In [1]:
import pandas as pd
import numpy as np

# Load the CSV file, replacing commas with dots and ensuring all data is treated as floats
prices = pd.read_csv('Extrarow_with_empty_rows_update.csv', sep=';', dtype=str)

# replace NaNs with zeros
prices = prices.fillna(0)

for i in range(1, len(prices), 2):
    prices.at[i, 'Code'] = 'Rm'



In [ ]:

# Convert the 'T-1Y' and 'T+1Y' columns to datetime format
prices['T-1Y'] = pd.to_datetime(prices['T-1Y'], format='%d/%m/%Y', errors='coerce')
prices['T+1Y'] = pd.to_datetime(prices['T+1Y'], format='%d/%m/%Y', errors='coerce')

# Subtract 5 days from the dates in the 'T-1Y' and 'T+1Y' columns
prices['T-1Y'] = prices['T-1Y'] - pd.Timedelta(days=5)
prices['T+1Y'] = prices['T+1Y'] - pd.Timedelta(days=5)

print(prices.head())



In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

prices = prices.drop('SPLIT', axis=1)

# Define the risk-free rate
risk_free_rate = 0.002

prices['date'] = pd.to_datetime(prices['Date'], format='%d/%m/%Y', errors='coerce')
print(prices)

print(prices.head())

# Create a DataFrame to store the abnormal returns
abnormal_returns = pd.DataFrame()

# Loop through each firm's data (assuming firm data is every other row, and 'Rm' is the row below it)
for i in range(0, len(prices), 2):  # Step by 2 to skip 'Rm' rows
    firm_name = prices.iloc[i, 0]  # Assuming firm ticker is in the first column
    firm_returns = prices.iloc[i, 7:].astype(float)  # Assuming returns start after column 6
    market_returns = prices.iloc[i + 1, 7:].astype(float)  # 'Rm' is the row directly below the firm
    
    # Convert firm and market returns to excess returns (return - risk-free rate)
    firm_excess_returns = firm_returns - risk_free_rate
    market_excess_returns = market_returns - risk_free_rate
    
    # Combine firm and market returns into a DataFrame to remove NaNs and inf values
    combined_data = pd.DataFrame({
        'firm_excess_returns': firm_excess_returns,
        'market_excess_returns': market_excess_returns
    })
    
    # Drop rows with NaN or infinite values
    combined_data = combined_data.replace([np.inf, -np.inf], np.nan).dropna()

    # Check if there is enough data after cleaning
    if len(combined_data) < 2:
        print(f"Not enough data for firm {firm_name}, skipping...")
        continue
    
    # Perform regression (market excess return as independent variable, firm excess return as dependent variable)
    X = sm.add_constant(combined_data['market_excess_returns'])  # Add intercept (alpha)
    Y = combined_data['firm_excess_returns']
    model = sm.OLS(Y, X).fit()
    
    # Extract alpha and beta from the regression
    alpha = model.params['const']
    beta = model.params['market_excess_returns']  # Beta corresponds to market return slope
    
    # Calculate abnormal returns: R_it = alpha + beta * R_m
    abnormal_return = alpha + beta * combined_data['market_excess_returns']
    
    # Add the abnormal returns to the DataFrame
    abnormal_returns[firm_name] = round(abnormal_return, 4)

    # --- PLOTTING ---
    # if i % 100 == 0:
    #     plt.figure(figsize=(8, 6))
        
    #     # Scatter plot: Firm excess returns vs market excess returns
    #     plt.scatter(combined_data['market_excess_returns'], combined_data['firm_excess_returns'], label='Data Points', color='blue')
        
    #     # Plot fitted line: alpha + beta * market_excess_return
    #     fitted_line = alpha + beta * combined_data['market_excess_returns']
    #     plt.plot(combined_data['market_excess_returns'], fitted_line, label=f'Fitted Line (α={alpha:.4f}, β={beta:.4f})', color='red')
        
    #     # Add labels and title
    #     plt.xlabel('Market Excess Returns')
    #     plt.ylabel('Firm Excess Returns')
    #     plt.title(f'{firm_name} - Excess Returns vs Market Returns')
    #     plt.legend()
        
    #     # Show the plot
    #     plt.show()

# Save the abnormal returns DataFrame to a CSV file
abnormal_returns.to_csv('abnormal_returns.csv', sep=';')

# Optional: display a preview of the abnormal returns
print(abnormal_returns.head())
